In [1]:
import pyspark
sc = SparkContext.getOrCreate()

In [10]:
def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)


lines = sc.textFile("temp.csv")
parsedLines = lines.map(parseLine)
minTemps = parsedLines.filter(lambda x: "TMIN" in x[1])
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
minTemps = stationTemps.reduceByKey(lambda x, y: min(x,y))
minTemps_results = minTemps.collect()

maxTemps = parsedLines.filter(lambda x: "TMAX" in x[1])
stationTemps = maxTemps.map(lambda x: (x[0], x[2]))
maxTemps = stationTemps.reduceByKey(lambda x, y: max(x,y))
maxTemps_results = maxTemps.collect();

print("-----------MIN TEMP-----------------------")
for minTemp in minTemps_results:
    print(minTemp[0] + "\t{:.2f}F".format(minTemp[1]))
print("-----------MAX TEMP-----------------------")
for maxTemp in maxTemps_results:
    print(maxTemp[0] + "\t{:.2f}F".format(maxTemp[1]))


-----------MIN TEMP-----------------------
ITE00100554	5.36F
EZE00100082	7.70F
-----------MAX TEMP-----------------------
ITE00100554	90.14F
EZE00100082	90.14F
